In [ ]:
%matplotlib inline 

from dolfin import *
import matplotlib.pyplot as plt

import numpy as np

plt.close()
T_final = 10.0 # final time
dt = 0.1
num_steps =int(T_final/dt)     # number of time steps

# Sub domain for Dirichlet boundary condition
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (abs(x[0]) < DOLFIN_EPS) or (abs(x[0] - 1.0) < DOLFIN_EPS) 

# Create mesh and define function space
mesh = UnitSquareMesh(100, 100)
File("mesh.pvd") << mesh

V = FunctionSpace(mesh, "CG", 1)

rho_0 = 800 #original 800
c_0 = 830 #original 830
#convection coefficient. 
h_conv = 11.0
# kappa not in use with steady state solutions 
kappa_0 = 2.03

# Define function space and basis functions
V = FunctionSpace(mesh, "CG", 2)
u = TrialFunction(V)
v = TestFunction(V)

u_init = Constant('313.0')
u_n = interpolate(u_init, V)
T_0 = 313.0

sigma = 1.387 * pow(10,-23)
epsilon = 0.73

g2 = Expression("((abs(x[1] - 1.0) < 1e-4) && (x[0]>0.49) && (x[0]<0.51)) ? 600000.0:0.0", degree=1)

# Time-stepping
u = Function(V)
t = 0

print('Final time: ' + str(T_final))

for n in range(num_steps):
    
    F = (rho_0*c_0*u*v*dx + kappa_0*dt*dot(grad(u), grad(v))*dx() + kappa_0*h_conv*(u)*v*dt*ds() - rho_0*c_0*u_n*v*dx() - kappa_0*g2*v*dt*ds() - kappa_0*h_conv*dt*(T_0)*v*ds()
            + kappa_0*epsilon*sigma*dt*u**4*v*ds() - kappa_0*epsilon*dt*sigma*T_0**4*v*ds())
    # Update current time
    t += dt

    solve(F == 0, u, solver_parameters={"newton_solver":{"relative_tolerance":1e-6},
                                     "newton_solver":{"maximum_iterations":500}})
    
    # Update previous solution
    u_n.assign(u)
    
    print('Current time ' + str(t))

# Plot solution
plt.figure()
p = plot(u)

plt.colorbar(p)
plt.interactive(True)


plt.figure()
plot(grad(u), title="Solution gradient")

plt.show()

